In [1]:
# This Cell to be executed for installing the required libraries if not already present.

# Install geopy which is python library providing geocoding and reverse geocoding services
# In this model, geopy allow to convert the latitudes and longitudes to human readable
# location and city names. Geopy requests is used to make external web calls like weather data , past fire
# info etc. Installing the library so these features can be used.
!pip install geopy requests --quiet

# Install ipyleaflet branca which allows to create interactive maps in
# this notebook
!pip -q install ipyleaflet branca --quiet


In [2]:
# This is the main program modules,which will display a map to the user, allow the user to drop a location where there is fire and select a Radius
# for the model's consideration. With this info, the model will predict the fire spread based on the current weather conditions, wind direction, previous
# fire occurance, and output the direction of fire and a prediction of the city fire is expected to be in the next 2 hrs.

# For interacting with the Operating system to fetch environment variables  FIRMS_MAP_KEY and OPENWEATHER_API_KEY
import os
# For interacting with stream classes
import io
# For Data analysis and manipulation
import pandas as pd
# For HTTP Requests to call external api's which will return a Json object with the required info like
# current weather conditions, past fire info etc from the external websites like OpenWeatherMap and
# Earthdata
import requests
# For geocoding Address to coordinates and vice versa
from geopy.geocoders import Nominatim
# For calculating distance between two locations
from geopy.distance import geodesic
# For math functions
import math
# For calculating the circular distance between two points given a radius
from math import radians, cos, sin, asin, sqrt
# For creating and manipulating grids for the heatmap
import numpy as np
# For storing and iterating past fire info
import pandas as pd
# Below packages and classes are for interactive controls like sliders, buttons etc in Jupyter notebooks, displaying shapes,
# calculating the distance between two locations etc.
import ipywidgets as widgets
from ipyleaflet import Map, Marker, Circle, Polygon, Heatmap, LayersControl, WidgetControl
from IPython.display import display, clear_output
from geopy.distance import geodesic

# For setting Environment variables to Keys to access FIRMS API's and Open weather API's
# Add your API key here - removing the keys before adding to github for security reasons.
os.environ["FIRMS_MAP_KEY"] = "GetMAPkey from the URL" # https://firms.modaps.eosdis.nasa.gov/api/
os.environ["OPENWEATHER_API_KEY"] = "GetAPI Key from the URL" # https://home.openweathermap.org/api_keys

# Initialize the geocoder with a 5 sec time out to overide the default 1 sec time out of Nominatim
geocoder = Nominatim(user_agent="fire_spread_app", timeout=5)

# Function to compute the haversine distance. shortest path between two given points on earth.
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth's radius in miles
    # convert latitudes to radians
    lat1_rad = math.radians(lat1)
    lat2_rad = math.radians(lat2)

    # Differences in latitude and longitude (in radians)
    delta_lat = math.radians(lat2 - lat1)
    delta_lon = math.radians(lon2 - lon1)

    # Apply the Haversine formula
    # 'a' is the square of half the chord length between the points
    a = (
        math.sin(delta_lat / 2) ** 2
        + math.cos(lat1_rad) * math.cos(lat2_rad)
        * math.sin(delta_lon / 2) ** 2
    )

    # angular distance in radians
    angular_distance = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * angular_distance


# Function to reach the destination point including the angle of direction
def destination_point(lat, lon, direction_degrees, distance_miles):
    dest = geodesic(miles=distance_miles).destination((lat, lon), direction_degrees)
    return dest.latitude, dest.longitude

# Function to build the list of coordinate points defining the wedge drawn on
# the map pointing in the wind direction. half_angle_deg is assumed to be 25 deg which gives
# a 50 deg wide cone centered at the location of fire.
def make_spread_wedge(lat, lon, direction_degrees, max_dist_miles, half_angle_deg=25, steps=20):
    left = (direction_degrees - half_angle_deg) % 360
    right = (direction_degrees + half_angle_deg) % 360
    pts = [(lat, lon)]
    for b in np.linspace(left, right, steps):
        pts.append(destination_point(lat, lon, b, max_dist_miles))
    pts.append((lat, lon))
    return pts

# Function to estimate how likely a given point  is to be affected
# by a fire starting at (ign_lat, ign_lon) under current wind conditions
# and within a certain radius. It combines three ideas: distance from the ignition,
# alignment with the wind direction, and proximity to recent fire detections.

def risk_score(point_lat, point_lon, ign_lat, ign_lon, wind_direction, wind_speed_m_s, radius_miles, past_fires):
    d = haversine(ign_lat, ign_lon, point_lat, point_lon)
    # point beyond chosen radius is out of scope
    if d > radius_miles:
        return 0.0
    # compass direction from location of fire (ignition) to a given point
    y = math.sin(math.radians(point_lon - ign_lon)) * math.cos(math.radians(point_lat))
    x = (math.cos(math.radians(ign_lat)) * math.sin(math.radians(point_lat))
         - math.sin(math.radians(ign_lat)) * math.cos(math.radians(point_lat))
         * math.cos(math.radians(point_lon - ign_lon)))
    compassdirection_to_point = (math.degrees(math.atan2(y, x)) + 360) % 360
    # Wind alignment factor
    ang_diff = abs((compassdirection_to_point - wind_direction + 180) % 360 - 180)
    alignment = max(0.0, 1.0 - (ang_diff / 180.0))
    # Distance decay factor
    dist_factor = max(0.0, 1.0 - (d / radius_miles))
    # Boost if near a recent fire (within 5 miles approximation)
    boost = 0.0
    for _, row in past_fires.iterrows():
        if haversine(row['lat'], row['lon'], point_lat, point_lon) < 5.0:
            boost = 0.3
            break
    return min(1.0, alignment * dist_factor + boost)

# Function for getting  wind conditions for a given location from OpenWeatherMap
# OpenWeatherMap returns wind data under the "wind" key, with speed in meters per
# second and deg is direction in degrees.
def get_weather_data(lat, lon, api_key=None):
    if api_key is None:
        api_key = os.environ.get("OPENWEATHER_API_KEY")
    if api_key is None:
        raise ValueError("No API key provided for weather data.")

    import requests
    url = (
        f"https://api.openweathermap.org/data/2.5/weather?"
        f"lat={lat}&lon={lon}&units=metric&appid={api_key}"
    )
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    # extract wind speed and direction
    return {
        "wind_speed": data.get("wind", {}).get("speed", 0.0),
        "wind_deg": data.get("wind", {}).get("deg", 0.0),
    }

# Function for getting past fire data using NASA FIRMS assunmption past 7 days
def get_past_fire_data(lat, lon, radius_miles, days=7):
    key = os.environ.get("FIRMS_API_KEY")
    url = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{key}/VIIRS_SNPP_NRT/{lon},{lat},{radius_miles},{days}"
    resp = requests.get(url, timeout=20)
    resp.raise_for_status()
    df = pd.read_csv(io.StringIO(resp.text))
    if df.empty:
        return pd.DataFrame(columns=["lat", "lon"])
    # keeping the output to lowercase
    df.columns = [c.lower() for c in df.columns]
    # Check for 'latitude'/'longitude' or 'lat'/'lon' as the dataset could have either
    if 'latitude' in df.columns and 'longitude' in df.columns:
        return df[['latitude','longitude']].rename(columns={'latitude':'lat','longitude':'lon'})
    elif 'lat' in df.columns and 'lon' in df.columns:
        return df[['lat','lon']]
    else:
        # If the expected columns are missing, return empty
        return pd.DataFrame(columns=['lat','lon'])

# Main interactive MAP Function
# This Function sets up an interactive map with a radius slider, processes
# the user clicks to predict fire spread, and displays the predictive model result
# which includes the current fire location, wind speed, predicted fire spread and
# and area with a pointer to the direction and next city in 2 hrs time frame.

def build_fire_spread_map():
    m = Map(center=(37.0, -119.5), zoom=6)
    marker = circle = wedge_layer = heat_layer = None
    out = widgets.Output()

    # create slider (hide the built-in description column)
    radius_slider = widgets.FloatSlider(
      value=20.0, min=1.0, max=100.0, step=1.0, description='',
      style={'description_width': '0px'}
    )

    # This is done so that the text can be displayed as black colored
    radius_label = widgets.HTML()

    def update_label(change=None):
      v = radius_slider.value if change is None else change["new"]
      radius_label.value = (
        f"<div style='color: #000 !important; font-weight: 600;'>"
        f"Radius: {v:.1f} miles"
        f"</div>"
      )

    update_label()  # set initial label value
    radius_slider.observe(update_label, names='value')

    # Combine label + slider
    slider_box = widgets.VBox(
    [radius_label, radius_slider],
    layout=widgets.Layout(
        padding="6px",
        border="1px solid rgba(0,0,0,0.25)",
        background_color="white"
      )
    )

    # Nested function which responds to click events
    def handle_click(**kwargs):
      nonlocal marker, circle, wedge_layer, heat_layer
      if kwargs.get('type') != 'click':
        return
      coords = kwargs.get('coordinates') or kwargs.get('latlng')
      if not coords:
        return
      lat, lon = coords

      # Reverse Geocode the Fire location
      try:
         ign_location = geocoder.reverse((lat, lon), exactly_one=True)
         ignition_city = ign_location.address if ign_location else "unknown"
      except Exception:
        ignition_city = "unknown"

       # Clear existing layer
      for layer in (marker, circle, wedge_layer, heat_layer):
        if layer:
            m.remove_layer(layer)

      # Add Marker
      marker = Marker(location=(lat, lon))
      m.add_layer(marker)

      # Fetching Weather Data
      weather = get_weather_data(lat, lon)
      wind_speed, wind_deg = weather['wind_speed'], weather['wind_deg']

      # Fetching past fire info
      past = get_past_fire_data(lat, lon, radius_slider.value)

      # Drawing the spread wedge
      wedge_points = make_spread_wedge(
          lat, lon, wind_deg, radius_slider.value, half_angle_deg=25, steps=20
      )
      wedge_layer = Polygon(
          locations=wedge_points, color="red", fill_color="red", fill_opacity=0.2
      )
      m.add_layer(wedge_layer)

     # Heatmap risk grid, 69 miles = 1 degree of latitude to convert miles to degrees.
      # 25 is an assumption evenly spaced value
      grid_lats = np.linspace(
          lat - radius_slider.value / 69.0, lat + radius_slider.value / 69.0, 25
      )
      grid_lons = np.linspace(
          lon - radius_slider.value / 69.0, lon + radius_slider.value / 69.0, 25
      )
      heat_data = []
      for glat in grid_lats:
        for glon in grid_lons:
            if haversine(lat, lon, glat, glon) <= radius_slider.value:
                r = risk_score(
                    glat, glon, lat, lon, wind_deg, wind_speed, radius_slider.value, past
                )
                if r > 0:
                    heat_data.append([glat, glon, r])
      if heat_data:
        heat_layer = Heatmap(
            locations=heat_data, radius=20, blur=15, min_opacity=0.1
        )
        m.add_layer(heat_layer)

      # Circle outlining the Radius , 1 mile = 1609.34 meters
      circle = Circle(
        location=(lat, lon),
        radius=int(radius_slider.value * 1609.34),
        color="blue",
        fill_opacity=0.1,
      )
      m.add_layer(circle)

      # Computing direction and estimated spread 2 hrs estimate
      compass_dirs = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
      idx = int((wind_deg + 22.5) // 45) % 8
      compass = compass_dirs[idx]
      wind_speed_mph = wind_speed * 2.23694
      est_distance_miles = wind_speed_mph * 2  # 2‑hour estimate

      # Compute destination point at the edge of the radius
      dest_lat, dest_lon = destination_point(lat, lon, wind_deg, radius_slider.value)

      # Reverse geocode to find nearest city
      try:
        location = geocoder.reverse((dest_lat, dest_lon), exactly_one=True)
        nearest_city = location.address if location else "unknown"
      except Exception:
        nearest_city = "unknown"

      # Output to display
      with out:
        clear_output()
        print(f"Fire location co-ordinates: ({lat:.4f}, {lon:.4f})")
        print(f"Fire location: {ignition_city}")
        print(f"Wind speed: {wind_speed:.2f} m/s, direction: {wind_deg:.0f}° ({compass})")
        print(f"{len(past)} past fire detections within {radius_slider.value:.1f} miles")
        print(f"Expected spread direction: {compass}")
        print(f"Estimated potential spread in 2 hours: ~{est_distance_miles:.1f} miles (assuming constant wind)")
        print(f"Nearest city along spread direction: {nearest_city}")

    # Attaching Click Handlers and Controls
    m.on_interaction(handle_click)
    m.add_control(WidgetControl(widget=slider_box, position='topright'))
    m.add_control(LayersControl(position='topright'))
    display(m, out)

# Launch the MAP
build_fire_spread_map()


Map(center=[37.0, -119.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

Output()